In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2
import os

In [4]:
import os
print(os.listdir("/gdrive/My Drive/projetum/Banana"))


['overripe', 'ripe', 'rotten', 'unripe']


In [5]:
DATADIR = '/gdrive/My Drive/projetum/Banana'
CATEGORIES=["overripe","ripe","rotten","unripe"]


In [6]:
IMG_SIZE = 100

In [7]:
all_data =[]

def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                print("hata")
create_all_data()

In [8]:
print("toplam görüntü sayısı: ",len(all_data))

toplam görüntü sayısı:  13478


In [9]:
X = []
y = []

for categories, label in all_data:
  X.append(categories)
  y.append(label)

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [11]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [12]:
print(y_test)

[1 1 3 ... 2 1 1]


In [13]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(8490, 100, 100, 3)
(4044, 100, 100, 3)
(944, 100, 100, 3)
(8490,)
(4044,)
(944,)


In [14]:

import tensorflow as tf

train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=4)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=4)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=4)

In [15]:
print(y_test)

[1 1 3 ... 2 1 1]


In [16]:
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(64, (3,3), padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(4))
model.add(Activation('softmax'))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [17]:
from tensorflow import keras

callback_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor = 'val_accuracy', save_best_only=True, verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=3)
]

In [18]:
history = model.fit(x_train, train_yCl,
                    batch_size=64,
                    validation_data = (x_val, valid_yCl),
                    callbacks = callback_list,
                    epochs = 50
                    )

Epoch 1/50
133/133 [==============================] - ETA: 0s - loss: 0.9267 - accuracy: 0.7563
Epoch 1: val_accuracy improved from -inf to 0.30508, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


133/133 [==============================] - 30s 143ms/step - loss: 0.9267 - accuracy: 0.7563 - val_loss: 7.4009 - val_accuracy: 0.3051
Epoch 2/50
133/133 [==============================] - ETA: 0s - loss: 0.3552 - accuracy: 0.8702
Epoch 2: val_accuracy improved from 0.30508 to 0.32733, saving model to model.h5
133/133 [==============================] - 17s 125ms/step - loss: 0.3552 - accuracy: 0.8702 - val_loss: 3.5067 - val_accuracy: 0.3273
Epoch 3/50
133/133 [==============================] - ETA: 0s - loss: 0.2546 - accuracy: 0.9075
Epoch 3: val_accuracy improved from 0.32733 to 0.62076, saving model to model.h5
133/133 [==============================] - 17s 126ms/step - loss: 0.2546 - accuracy: 0.9075 - val_loss: 1.1934 - val_accuracy: 0.6208
Epoch 4/50
133/133 [==============================] - ETA: 0s - loss: 0.2246 - accuracy: 0.9184
Epoch 4: val_accuracy did not improve from 0.62076
133/133 [==============================] - 16s 119ms/step - loss: 0.2246 - accuracy: 0.9184 - val

In [19]:
score_train = model.evaluate(x_train, train_yCl)
print("Validation Accuracy: ", score_train[1])

score_test = model.evaluate(x_test, test_yCl)
print("Validation Accuracy: ", score_test[1])

score_valid = model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

266/266 [==============================] - 5s 15ms/step - loss: 0.2862 - accuracy: 0.9241
Validation Accuracy:  0.9241460561752319
127/127 [==============================] - 2s 17ms/step - loss: 0.5172 - accuracy: 0.8808
Validation Accuracy:  0.8808110952377319
30/30 [==============================] - 1s 27ms/step - loss: 0.6034 - accuracy: 0.8708
Validation Accuracy:  0.8707627058029175


In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Test seti üzerinde tahmin yapma
Y_pred = model.predict(x_test, steps=len(x_test))
y_pred = np.argmax(Y_pred, axis=1)

4044/4044 [==============================] - 9s 2ms/step


In [21]:
print(y_pred)

[1 1 3 ... 2 1 1]


In [22]:
# Gerçek etiketleri alma
y_true = y_test

In [23]:
print(y_test)

[1 1 3 ... 2 1 1]


In [24]:
# Accuracy, Precision, Recall, F1 Score hesaplama
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [25]:
# Karmaşıklık Matrisi
conf_matrix = confusion_matrix(y_test, y_pred)

In [26]:

# Classification Report
class_report = classification_report(y_test, y_pred)


In [27]:

# Sonuçları gösterme
print("Test Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Test Accuracy: 0.880811078140455
Precision: 0.8922518116604764
Recall: 0.880811078140455
F1 Score: 0.8792091708637824
Confusion Matrix:
[[ 561   38  223    0]
 [  15 1127   51    0]
 [  13   47 1285   13]
 [   2   10   70  589]]
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.68      0.79       822
           1       0.92      0.94      0.93      1193
           2       0.79      0.95      0.86      1358
           3       0.98      0.88      0.93       671

    accuracy                           0.88      4044
   macro avg       0.91      0.86      0.88      4044
weighted avg       0.89      0.88      0.88      4044

